In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import geemap
import ee
Map=geemap.Map()
Map 

In [ ]:
# 岳阳市
# china_city = ee.FeatureCollection('users/311605001111/ChinaCity')
# roi = china_city.filter(ee.Filter.eq('市','青岛市'))
roi = ee.Geometry.Rectangle([119.8904, 35.6035,121.3404, 36.4001])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [ ]:
# sentinel2 数据
def maskS2clouds(image):
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(mask).divide(10000)
visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

sentinel_images = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(roi) \
    .filterDate('2021-06-15', '2021-06-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .map(maskS2clouds)
Map.addLayer(sentinel_images.mosaic().clip(roi),visualization,'sentinel image')
print(sentinel_images.size().getInfo())

In [ ]:
# landsat8 数据
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2021-06-01', '2021-06-30') \
    .filter(ee.Filter.lt('CLOUD_COVER',20))\
    .map(maskL8sr)
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')
print(landsat_images.size().getInfo())